In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
browser = webdriver.Chrome()

In [3]:
url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=002&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
browser.get(url)
time.sleep(1)

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [5]:
# 남성 클릭
btn_list = browser.find_elements('css selector', 'div.sc-pvystx-0.iRAZYo > button')
browser.execute_script("arguments[0].click();", btn_list[1])


In [6]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

# total page
total_page = soup.select('div.pagingNumber-box.box > span > span.totalPagingNum')[0].text
time.sleep(2)
page_count = 1
results = []
max_retries = 3  # 재시도 횟수 제한
rank = 1

timeout_threshold = 60
# 마지막으로 성공적인 페이지 스크래핑 시간을 추적하는 변수
last_successful_time = time.time()

while(1):
    if page_count == 1 :

        time.sleep(2)  # 페이지 로딩 대기
        soup = BeautifulSoup(browser.page_source, "html.parser")
        item_list = soup.select('#goodsRankList > li > div.li_inner')

        for item in item_list:
            brand_list = item.select('div.article_info > p.item_title > a')[0].text
            raw_title = item.select('div.article_info > p.list_info > a')[0].text.replace('\n', '').strip()
            price_list = item.select('div.article_info > p.price')[0].text.replace('\n', '').strip()
            split_price = price_list.split()
            img_list = item.select('div.list_img > a > img.lazyload.lazy')[0]['data-original']
            link_list = item.select('div.list_img > a')[0]['href']
            
            if len(split_price) >= 2:
                price_original = split_price[0]
                price_now = split_price[1]
            else:
                price_original = split_price[0]
                price_now = split_price[0]

            if raw_title.startswith('('):
                title_list = raw_title.split(')', 1)[1].strip()
            elif raw_title.startswith('['):
                title_list = raw_title.split(']', 1)[1].strip()
            elif '배송' in raw_title:
                title_list = raw_title.split('송', 1)[1].strip()
            else:
                title_list = raw_title
            
            data = [rank, brand_list, title_list, price_original, price_now, link_list ,img_list]
            results.append(data)
            rank += 1
            last_successful_time = time.time()
        page_count += 1
        
    else:
        while page_count < int(total_page):
            retries = 0
            while retries < max_retries:
                try:
                    # 페이지 버튼 클릭을 위한 반복문
                    for i in range(3, 13):
                        page_btn_list = browser.find_elements('css selector', 'div.sorter-box.box > div > div > a')
                        if i >= len(page_btn_list):
                            break  # 페이지 버튼이 더 이상 없으면 반복문 종료

                        page_btn = page_btn_list[i]
                        browser.execute_script("arguments[0].click();", page_btn)
                        time.sleep(2)  # 페이지 로딩 대기

                        soup = BeautifulSoup(browser.page_source, "html.parser")
                        item_list = soup.select('#goodsRankList > li > div.li_inner')

                        for item in item_list:
                            brand_list = item.select('div.article_info > p.item_title > a')[0].text
                            raw_title = item.select('div.article_info > p.list_info > a')[0].text.replace('\n', '').strip()
                            price_list = item.select('div.article_info > p.price')[0].text.replace('\n', '').strip()
                            split_price = price_list.split()
                            img_list = item.select('div.list_img > a > img.lazyload.lazy')[0]['data-original']
                            
                            if len(split_price) >= 2:
                                price_original = split_price[0]
                                price_now = split_price[1]
                            else:
                                price_original = split_price[0]
                                price_now = split_price[0]

                            if raw_title.startswith('('):
                                title_list = raw_title.split(')', 1)[1].strip()
                            elif raw_title.startswith('['):
                                title_list = raw_title.split(']', 1)[1].strip()
                            elif '배송' in raw_title:
                                title_list = raw_title.split('송', 1)[1].strip()
                            else:
                                title_list = raw_title
                            
                            data = [rank, brand_list, title_list, price_original, price_now, link_list ,img_list]
                            results.append(data)
                            rank += 1
                            last_successful_time = time.time()
        #                     print(brand_list, title_list, price_original, price_now)
                        page_count += 1
                        print(page_count)
                    break  # 현재 페이지 처리 완료 후, break로 내부 while 루프 종료

                except Exception as e:
                    print(f"Error occurred: {e}, retrying...")
                    retries += 1
                    time.sleep(5)  # 에러 발생 시 잠시 기다린 후 재시도

            if retries == max_retries:
                print("Max retries reached. Moving to next page.")
                page_count += 1  # 최대 재시도 횟수에 도달하면 다음 페이지로 넘어감

            if page_count >= int(total_page):
                break  # 전체 페이지 처리 완료 후, 외부 while 루프 종료
            
            if time.time() - last_successful_time > timeout_threshold:
                print("타임아웃에 도달했습니다. 루프를 종료합니다.")
                break
        break
# # 결과 출력 또는 저장
print(results)


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
[[1, '스파오', '베이직 푸퍼_SPJPD4TC11', '69,900원', '64,900원', 'https://www.musinsa.com/app/goods/2037173?loc=goods_rank', 'https://image.msscdn.net/images/goods_img/20210722/2037173/2037173_16976143519631_125.jpg'], [2, '드로우핏', '프리미어 퓨어 캐시미어 발마칸 코트 [BLACK]', '308,000원', '246,400원', 'https://www.musinsa.com/app/goods/2098325?loc=goods_rank', 'https://image.msscdn.net/images/goods_img/20210830/2098325/2098325_16970092892875_125.jpg'], [3, '비슬로우', 'OVERSIZED CASHMERE BALMACAAN COAT BLACK', '329,000원', '263,200원', 'https://www.musinsa.com/app/goods/2782764?loc=goods_rank', 'https://image.msscdn.net/images/goods_img/20220908/2782764/2782764_16966495820166_125.jpg'], [4, '무신사 스탠다드', '데일리 푸퍼 숏 패딩 재킷 [블랙]', '6

In [8]:
import pandas as pd
df = pd.DataFrame(results)
df.columns = ['랭킹', '브랜드', '상품명', '기존가격', '현재가격', '사이트' ,'이미지링크']
df.to_excel('./data/Musinsa_outer.xlsx',index=False)